# シンプルな DNN モデルを Keras で実装する

このノートブックでは、 Keras のパイプラインで ML データセットを読み込み、その後 ニューヨークのタクシー乗車料金を予測する Keras の DNN モデルを実装します。

### 目的
1. `tf.data` を利用して CSV ファイルを読み込む方法を確認する
2. DNN アーキテクチャの入力層、隠れ層、出力層を設計する
3. DNN アーキテクチャを確認、可視化する
4. モデルをローカルでトレーニングし、損失カーブを可視化する

## 環境の準備

In [ ]:
import os, json, math, subprocess
import numpy as np
import shutil
import tensorflow as tf
print("TensorFlow version: ",tf.version.VERSION)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # SET TF ERROR LOG VERBOSITY

## CSV ファイルを指定する

ここでは、最初のノートブックで作成したデータを読み込んでいきましょう。

In [ ]:
!ls -l ../data/*.csv
!head ../data/taxi-train*

## tf.data を利用して CSV ファイルを読み込む

`0_explore.ipynb` で作成したデータを仕様しましょう。このデータは `./data`内に格納されています。

まずは、すべてのデータ列名、予測するデータ列（ラベル）名、そしてデフォルトの値を定義しましょう。

In [ ]:
CSV_COLUMNS  = ['fare_amount',
            'dayofweek',
            'hourofday',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
           ]
LABEL_COLUMN = 'fare_amount'
DEFAULTS     = [[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]]

次に、使用する特徴量と予測するラベルを定義し、トレーニング用データセットを読み込みます。

In [ ]:
def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label  # features, label

# load the training data
def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS, num_epochs=None)
    else:
        dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS, num_epochs=1)
        
    dataset = dataset.map(features_and_labels).prefetch(1) # take advantage of multi-threading; 1=AUTOTUNE
    return dataset

## Keras で DNN モデルを作成する

では、 Deep Neural Network (DNN) のモデルを Keras で実装し、入力層と隠れ層を指定しましょう。<br>
そして DNN のアーキテクチャを出力、可視化してみましょう。

In [ ]:
## Build a simple Keras DNN using its Functional API
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true))) 

def build_dnn_model():
    INPUT_COLS = ['dayofweek',
            'hourofday',
            'pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
           ]

    # input layer
    inputs = {
        colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32')
           for colname in INPUT_COLS
    }
    feature_columns = {
        colname : tf.feature_column.numeric_column(colname)
           for colname in INPUT_COLS
    }
    
    # the constructor for DenseFeatures takes a list of numeric columns
    # The Functional API in Keras requires that you specify: LayerConstructor()(inputs)
    dnn_inputs = tf.keras.layers.DenseFeatures(feature_columns.values())(inputs)

    # two hidden layers of [32, 8] just in like the BQML DNN
    h1 = tf.keras.layers.Dense(32, activation='relu', name='h1')(dnn_inputs)
    h2 = tf.keras.layers.Dense(8, activation='relu', name='h2')(h1)

    # final output is a linear activation because this is regression
    output = tf.keras.layers.Dense(1, activation='linear', name='fare')(h2)
    model = tf.keras.models.Model(inputs, output)
    model.compile(optimizer='adam', loss='mse', metrics=[rmse, 'mse'])
    return model

print("Here is our DNN architecture so far:\n")
model = build_dnn_model()
print(model.summary())

### DNN を可視化する

Keras の [plot_model](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/utils/plot_model) 機能を利用して、 DNN を可視化してみましょう。

In [ ]:
tf.keras.utils.plot_model(model, 'dnn_model.png', show_shapes=False, rankdir='LR')

## モデルをトレーニングする

モデルをトレーニングは、 [model.fit()](https://keras.io/models/model/#fit) を呼び出すだけで実行できます。

NUM_TRAIN_EXAMPLES はより大きい数値を使用するべきでしょう。<br>
小さいデータでのトレーニング/評価に基づいてモデルの質を推し量るのは限界があるためです。


In [ ]:
BATCH_SIZE = 32
NUM_TRAIN_EXAMPLES = 1000 * 5 # training dataset repeats, so it will wrap around
NUM_EPOCH = 10  # how many times to evaluate

trainds = load_dataset('../data/taxi-train*', BATCH_SIZE, tf.estimator.ModeKeys.TRAIN)
evalds = load_dataset('../data/taxi-valid*', BATCH_SIZE, tf.estimator.ModeKeys.EVAL)

steps_per_epoch = NUM_TRAIN_EXAMPLES // BATCH_SIZE 

history = model.fit(trainds, 
                    validation_data=evalds,
                    epochs=NUM_EPOCH, 
                    steps_per_epoch=steps_per_epoch)

### モデルの損失カーブを可視化する

続いて、matplotlib を使用して、トレーニングと評価の損失カーブを可視化してみましょう。

In [ ]:
# plot
import matplotlib.pyplot as plt
nrows = 1
ncols = 2
fig = plt.figure(figsize=(10, 5))

for idx, key in enumerate(['loss', 'rmse']):
    ax = fig.add_subplot(nrows, ncols, idx+1)
    plt.plot(history.history[key])
    plt.plot(history.history['val_{}'.format(key)])
    plt.title('model {}'.format(key))
    plt.ylabel(key)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left');

## モデルを使った推論をローカルで実行する

Keras を用いた推論は、 [model.predict()](https://keras.io/models/model/#predict) を呼び出し、特徴量を渡すことで実行することができます。

試しに、[ウォール街のチェイス銀行からブライアント公園までのルート](https://www.google.com/maps/dir/Chase+Bank,+45+Wall+St,+New+York,+NY+10005,+United+States/Bryant+Park,+New+York,+NY+10018,+United+States/data=!4m8!4m7!1m2!1m1!1s0x89c25a16d45a5027:0x215b141554f11385!1m2!1m1!1s0x89c259aae7a0b1bd:0xb49cafb82537f1a7!3e0?sa=X&ved=2ahUKEwjs9fnF8KzuAhUGE6YKHSQJDhcQ-A96BAgBEA4)の料金を調べてみましょう。

In [ ]:
model.predict({
    'pickup_longitude': tf.convert_to_tensor([-74.0098147]),
    'pickup_latitude': tf.convert_to_tensor([40.7060502]),
    'dropoff_longitude': tf.convert_to_tensor([-73.9833705]),
    'dropoff_latitude': tf.convert_to_tensor([40.7532916]),
    'hourofday': tf.convert_to_tensor([18.0]), 
    'dayofweek': tf.convert_to_tensor([1.0]), 
}, steps=1, batch_size=1)

しかし、もちろんこの方法は現実的ではありません。なぜなら、クライアントのメモリにモデルオブジェクトが置いてあることは稀だからです。<br>
モデルをファイルにエクスポートし、クライアント側はそのファイルからモデルを使用できるように設定する必要があります。

この機能については、次のノートブックで学習します。

Copyright 2019 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.